In [1]:
# %pip install dill
# %pip install nltk
# %pip install inflect
# %pip install pydantic

In [2]:
import sys
import openai

sys.path.insert(0, "../")
sys.path.insert(0, "../..")
from GenAgents.text_adventure_games.parsing import GptParser3
from GenAgents.test.game_setup import build_conference

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kylesullivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kylesullivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
game = build_conference(
    experiment_name="conference-test",
    experiment_id=1,
    num_characters=4,
    max_ticks=4,
    personas_path="game_personas",
    # random_placement=True,
)
game.give_hints = True
parser = GptParser3(game, verbose=False)
game.set_parser(parser)
parser.refresh_command_list()

try:
    game.game_loop()
except openai.APIConnectionError:
    print("Could not connect to OpenAI API")
finally:
    game.save_simulation_data()


GET OR CREATE BASE FACTS PLAYER DESCRIPTION: An quirky contestant that is must see TV on a reality show. True
RESPONSE: Name="Maximiliano 'Max' Ross" Age=27 Likes=['improv comedy', 'spontaneous dance parties', 'bizarre street art', 'creating absurd viral videos'] Dislikes=['predictability', 'rudeness', 'lack of creativity'] Occupation='Reality Show Contestant' Home_City='Austin, Texas'
Generated facts: {'Name': "Maximiliano 'Max' Ross", 'Age': 27, 'Likes': ['improv comedy', 'spontaneous dance parties', 'bizarre street art', 'creating absurd viral videos'], 'Dislikes': ['predictability', 'rudeness', 'lack of creativity'], 'Occupation': 'Reality Show Contestant', 'Home_City': 'Austin, Texas'}
Adding Maximiliano 'Max' Ross (group D)

GET OR CREATE BASE FACTS PLAYER DESCRIPTION: An quirky contestant that is must see TV on a reality show. True
RESPONSE: Name='Zane Carter' Age=28 Likes=['juggling', 'stand-up comedy', 'extreme sports'] Dislikes=['being serious', 'boring small talk', 'heights

In [4]:
import json
from GenAgents.text_adventure_games.parsing import Parser

experiment_id = "1"
experiment_name = "conference-test"

wrap_width = 120

def print_with_symbols(ratio, total_length, message, symbol):
    message_length = len(message) + 2  # Including the surrounding symbols
    num_symbols = (total_length - message_length) // (ratio + 1)  # Calculate number of symbols
    num_spaces = (total_length - (num_symbols * 2 + message_length)) // 2  # Calculate number of spaces
    print(f"{' ' * num_spaces}{symbol * num_symbols} {message} {symbol * num_symbols}\n")

# Read the JSONL file and print the 'message' field
with open(
    f"/Users/kylesullivan/Documents/Professional/LLM Agents/GenAgents/GenAgents/logs/{experiment_name}-{experiment_id}/gpt_call_{experiment_name}-{experiment_id}.jsonl",
    "r",
) as file:
    for i, line in enumerate(file):
        data = json.loads(line)
        message = data.get("messages", "No message found")
        character = data.get("character_name", "No character found")
        round = data.get("round", "No round found")
        tick = data.get("tick", "No tick found")

        # Calculate the number of symbols needed for formatting
        title = f"{character}: {round}.{tick}"
        symbols_needed = (wrap_width - len(title) - 2) // 2
        symbols = "=" * symbols_needed

        print(f"\n{symbols} {title} {symbols}\n")

        # Check if message is a list
        if isinstance(message, list):
            for msg in message:
                if isinstance(msg, tuple | list) and len(msg) == 2:
                    total_length = 120
                    ratio = 2
                    print_with_symbols(ratio, total_length, msg[0], '~')
                    print(Parser.wrap_text(msg[1]), "\n")
                else:
                    print("FAILURE:", type(msg), Parser.wrap_text(msg))
        else:
            print(Parser.wrap_text(message))

        print_with_symbols(ratio, total_length, "Response", "~")
        response_data = data.get("response", "No response found")

        # Check if response_data is a valid JSON string before loading
        if response_data and isinstance(response_data, str):
            try:
                response_json = json.loads(response_data)
                if isinstance(response_json, dict):
                    for key, value in response_json.items():
                        print(Parser.wrap_text(f"{key}:"), end="\n")
                        if (
                            isinstance(value, list)
                            and len(value) > 0
                            and isinstance(value[0], dict)
                        ):
                            for item in value:
                                for k, v in item.items():
                                    print(Parser.wrap_text(f"{k}:\n- {str(v)}\n"))
                        else:
                            print(Parser.wrap_text(f"- {str(value)}\n"))
                else:
                    print(Parser.wrap_text("Response is not a dictionary."))
            except json.JSONDecodeError:
                print(Parser.wrap_text("Error decoding JSON response."))
                print(Parser.wrap_text(response_data))
        else:
            print(Parser.wrap_text("No valid response data found."))


============================================= Maximiliano 'Max' Ross: 0.0 =============================================

                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ System ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

WORLD INFO:
You are seated at a conference meeting for Apple Inc. Q2 2025 Board Meeting, discussing Strategic Planning for
the Next Quarter and Review of Q1 2025 Results. Also present are: Zane Carter, Tyler 'T-Bone' Thompson, and
Vivian Hargrove. Having carefully read through the provided documents, you prepare talking points, arguments,
and counterarguments for the meeting. Once the meeting starts, you will be given a chance to speak. Your goal
is to present your case and sway the other conference members to your side.

PERSONAL INFO:
Maximiliano 'Max' Ross, a 27-year-old Reality Show Contestant. You are passionate about improv comedy,
spontaneous dance parties, bizarre street art. You have aversions to predictability, rudeness, lack of
creativity. Some key facts

In [5]:
def print_dialogue_from_log(experiment_id):
    """
    Reads a dialogue log from a specified JSON file and prints each line formatted.

    Args:
        experiment_id (str): The ID of the experiment to construct the log file path.
    """
    log_file_path = f"/Users/kylesullivan/Documents/Professional/LLM Agents/GenAgents/GenAgents/logs/conference-test-{experiment_id}/dialogue_log_conference-test-{experiment_id}.json"

    with open(log_file_path, "r") as dialogue_file:
        dialogue_lines = dialogue_file.readlines()

    for line in dialogue_lines:
        # Decode the escaped characters
        decoded_line = line.strip().encode('utf-8').decode('unicode_escape')
        print(Parser.wrap_text(decoded_line))
        print()  # Add a newline between responses

# Example usage
print_dialogue_from_log("1")

"Maximiliano 'Max' Ross: As we look at the strategic planning for the next quarter, I think it's important to
highlight the successes we've seen in Q1 2025. Our sales have increased by 15%, driven significantly by the
introduction of the new iPhone model. Additionally, our customer satisfaction ratings have reached an all-time
high, reflecting the hard work of our teams. However, we must also address some key areas for improvement. For
instance, our supply chain has faced delays that we need to rectify to ensure timely product launches. I've
got a few actionable suggestions on how we can streamline operations and enhance our marketing strategies to
capitalize on our momentum. By aligning our goals with these insights, I'm confident we can build a more
robust framework for success in the upcoming quarter."

"Vivian Hargrove: I totally agree with Max about the importance of building on our Q1 successes and addressing
the challenges we've faced. It's fantastic that sales have increased by